## TD学習 (Temporal Difference Learning)

source : Richard S. Sutton and Andrew G.Barto, 「強化学習」

---

### 強化学習の枠組み

強化学習の枠組みは、学習と意思決定を行う「エージェント」と  
それ以外のすべてから構成される「環境」の相互作用として表される。  
  
離散的な時間ステップ　$t=0,1,...$のあるステップtにおいて、  
エージェントは環境から状態$s_{t}$を受け取り、行動$a_{t}$を選択する。  
  
このとき、状態$s_{t}$から行動$a_{t}$への写像はエージェントの方策（policy）と呼ばれ、$\pi(s,a)$で表される。  

また、エージェントは、最終的に受け取る報酬を最大化することを目標として学習する。  
一般的には期待収益（expected return）を最大化するように設定される。  
  
各時間ステップtに受け取る報酬を$r_{t}$とするとき、  
最も単純な場合には、収益$R_{t}$は  
　$R(t) = r_{t+1} + r_{t+2} + ... + r_{T}  $  
として表される。ここでTは最終時間ステップである。（相互作用が離散的な場合）  
  
連続タスクにおいてはT=∞となりR(t)が発散しうるため、割引収益  
$$
\begin{align}
R_{t} &= r_{t+1} + \gamma r_{t+2} + \gamma^{2} r_{t+3} + ... \\  
　　　&= \sum_{k=0}^{\infty}\gamma^{k} r_{t+k+1} 
\end{align}
$$


を最大化するようにa(t)を選択する。  
ただし、γは割引率と呼ばれるパラメータで(0<=γ<=1)である。  

---

一般的な強化学習アルゴリズムは価値関数に基づく評価を行っている。  
方策πのもとでの状態sの価値Vπ(s)は、MDP（マルコフ決定過程）では  

$$
\begin{align}
V^{\pi}(s) = E_{\pi}\{R_{t} | s_{t}=s\} = E_{\pi}\{ \sum_{k=0}^{\infty}\gamma^{k}r_{t+k+1}|s_{t}=s\}
\end{align}
$$

と表される。関数 $ V^{\pi} $を方策πに対する状態価値観数と呼ぶ。
  
同様に、方策πのもとで状態sにおいて行動aを取ることの価値を $Q^{\pi}(s,a)$で表し、  
状態sで行動aを取り、その後に方策πに従った期待報酬として次のように定義する。  
$$
\begin{align}
Q^{\pi}(s,a)=E_{\pi}\{R_{t}|s_{t}=s, a_{t}=a\} = E_{\pi}\{\sum_{k=0}^{\infty}\gamma^{k}R_{t+k+1}|s_{t}=s, a_{t}=a\}
\end{align}
$$

---

任意のs,aが与えられたときの次に可能な各状態s'の確率を遷移確率と呼ぶ。  
$$
\begin{align}
P^{a}_{ss'} = Pr\{s_{t+1}=s'| s_{t}=s, a_{t}=a\}
\end{align}
$$
  
同様にして、次の報酬の期待値を次のように表す。
$$
\begin{align}
R^{a}_{ss'}=E\{r_{t+1}| s_{t}=s, a_{t}=a, s_{t+1}=s'\}
\end{align}
$$
  
    
強化学習と動的計画法で使われている価値観数は、  
任意の方策πと状態sに対して、sの価値と可能な後続状態群の価値との間に  
以下の整合性条件（consistency condition）がなりたち、これを$V^{\pi}$に対するBellman方程式という。  
  
$$
\begin{align}
V^{\pi}(s) &= E_{\pi}\{R_{t} | s_{t}=s\} \\
&= E_{\pi}\{ \sum_{k=0}^{\infty}\gamma^{k}r_{t+k+1}|s_{t}=s\} \\
&= E_{\pi}\{r_{t+1}+ \gamma\sum_{k=0}^{\infty}\gamma^{k}r_{t+k+2}|s_{t}=s\} \\
&= \sum_{a}\pi(s,a)\sum_{s'}P^{a}_{ss'} [ R^{a}_{ss'} + \gamma E_{\pi} \{ \sum_{k=0}^{\infty}\gamma^{k}r_{t+k+2}|s_{t+1}=s'\} ] \\
&= \sum_{a}\pi(s,a)\sum_{s'}P^{a}_{ss'}[R^{a}_{ss'}+\gamma V^{\pi}(s')]
\end{align}
$$


---

いま、すべての状態に対して、方策πの期待収益がπ'よりも良いか同じであるなら、  
πはπ'よりも良いか、同じであると定義する。  
つまり、すべての $ s \in S $ に対して、$ V^{\pi}(s) \leqq V^{\pi'}(s) $ であるなら、その時に限り $\pi \leqq \pi'$である。
  
これが１つの最適方策であり、すべての最適方策を$\pi^{*}$と記す。  
最適方策群は最適状態価値関数 $V^{*}(s) = \max_{\pi}V^{\pi}(s)$ を共有する。    


同様に、最適方策群は最適行動価値関数 $Q^{*}(s,a)=\max_{\pi}Q^{\pi}(s,a)$ を共有する。  
$V^{*}$を用いて$Q^{*}$を次のように書くことができる。  

$$
\begin{align}
Q^{*}(s,a)=E\{r_{t+1}+\gamma V^{*}(s_{t+1})|s_{t}=s,a_{t}=a\}
\end{align}
$$

$V^{*}$に対するBellman方程式を、Bellman最適方程式という。  
  
$$
\begin{align}
V^{*}(s) &= \max_{a \in A(s)}Q^{\pi^{*}}(s,a) \\
&= \max_{a}E_{\pi^{*}}\{R_{t} | s_{t}=s\} \\
&= \max_{a}E_{\pi^{*}}\{ \sum_{k=0}^{\infty}\gamma^{k}r_{t+k+1}|s_{t}=s\} \\
&= \max_{a}E_{\pi^{*}}\{r_{t+1}+ \gamma\sum_{k=0}^{\infty}\gamma^{k}r_{t+k+2}|s_{t}=s\} \\
&= \max_{a}E\{r_{t+1}+\gamma V^{*}(s_{t+1})|s_{t}=s,a_{t}=a\} \\
&= \max_{a}\sum_{s'}P^{a}_{ss'}[R^{a}_{ss'}+\gamma V^{*}(s')]
\end{align}
$$

$Q^{*}$に対するBellman最適方程式は次の通り。
$$
\begin{align}
Q^{*}(s) &= E\{r_{t+1}+\gamma \max_{a'}Q^{*}(s_{t+1},a')|s_{t}=s,a_{t}=a\} \\
&= \sum_{s'}P^{a}_{ss'}[R^{a}_{ss'}+\gamma \max_{a'} Q^{*}(s',a')]
\end{align}
$$

---

### TD学習

TD法は経験を用いて予測問題を解決し、方策πに従って経験をいくつか得ることで  
$V^{\pi}$ の推定値$V$を更新する手法の一つである。  
  
最も単純なTD法はTD(0)と呼ばれ、以下のようになる。
$$
\begin{align}
V(s_{t}) \leftarrow V(s_{t}) + \alpha [ r_{t+1}+\gamma V(s_{t+1}) - V(s_{t})]
\end{align}
$$
ここで$\alpha$はステップサイズパラメータである。

$V^{\pi}$に対するBellman方程式より
$$
\begin{align}
V^{\pi}(s) &= E_{\pi}\{R_{t} | s_{t}=s\} \\
&= E_{\pi}\{r_{t+1}+ \gamma\sum_{k=0}^{\infty}\gamma^{k}r_{t+k+2}|s_{t}=s\} \\
\therefore V^{\pi}(s) &= E_{\pi}\{r_{t+1}+ \gamma V^{\pi}(s_{t+1})|s_{t}=s \}
\end{align}
$$

としたとき、モンテカルロ法は前者の推定値を、動的計画法は後者の推定値を目標とする。  
TD法は両者を融合させたものである。  

### テーブル型 TD(0) アルゴリズム
Algorithm
>$V(s)$を任意に初期化し、$\pi$を評価対象の方策に初期化する  
>各エピソードに対して繰り返し：  
>　　$s$を初期化  
>　　エピソードの各ステップに対して繰り返し：  
>　　　　$ a \leftarrow s $に対して$\pi$で与えられる行動  
>　　　　行動$a$を取り、報酬$r$と次状態$s'$を観測する  
>　　　　$V(s) \leftarrow V(s) + \alpha[r+\gamma V(s')-V(s)]$  
>　　　　$s \leftarrow s'$  
>　　$s$が終端状態ならば繰り返しを終了  

TD予測法を制御問題に適用する方法について考える。

### Sarsa : 方策オン型TD制御
行動価値関数を学習するためにTD法を用いる。

Algorithm
>$Q(s,a)$を任意に初期化
>各エピソードに対して繰り返し：  
>　　$s$を初期化  
>　　$Q$から導かれる方策（εグリーディ方策など）を用いて、$s$で取る行動$a$を選択する  
>　　エピソードの各ステップに対して繰り返し：  
>　　　　行動$a$を取り、報酬$r$と次状態$s'$を観測する  
>　　　　$Q$から導かれる方策を用いて、$s'$での行動$a'$を選択する  
>　　　　$Q(s,a) \leftarrow Q(s,a) + \alpha[r+\gamma Q(s',a')-Q(s,a)]$  
>　　　　$s \leftarrow s'; a \leftarrow a';$  
>　　$s$が終端状態ならば繰り返しを終了 

In [ ]:
# WIP

# エージェントと環境との相互作用は離散的であり、エピソード的タスク群に分解されること、
# および行動の集合Aと状態の集合Sは有限の要素しか持たず、
# その数は学習開始時に既知であることを仮定する。
# policyとしてはe-greedyを用いる。

import numpy as np

class Q_function(object):
    def __init__(self, learning_rate, discount_rate, state_space_size, initial_value=0):
        self.learning_rate = learning_rate
        self.discount_rate = discount_rate
        self.state_space_size = state_space_size
        self.value_table = np.ones(self.state_space_size) * initial_value
    
    def estimate_value(self, state):
        return self.value_table[state]
    
    def update_value_table(self, state, reward, next_state):
        self.value_table[state] = self.value_table[state] + self.learning_rate * \
                                    (reward + self.discount_rate * self.value_table[next_state] - self.value_table[state])
    
    
class Policy_e_greed(object):
    def __init__(self, value_function, state_space_size, action_space_size, 
                 initial_play_count=0, reward_table=None, 
                 epsilon=0.1, min_choose=1):
        self.value_function = value_function
        self.state_space_size = state_space_size
        self.action_space_size = action_space_size
        self.play_count = initial_play_count
        self.min_choose = min_choose
        self.epsilon = epsilon
        
    def act_greedy(self):
        
    def act(self):
        
    def update_reward_table(self):
        
        
        
        
        
        # expected_reward[action_number] = (selected_count, sum of rewards, avarage of rewards)
        if expected_reward is None or expected_reward=={}:
            self.expected_reward = {}
            for i in range(action_space_size):
                self.expected_reward[i] = [0, 0., 0.]
        else:
            self.expected_reward = expected_reward
        

    def act_greedy(self):
        keys_of_less_selected = [key for key in self.expected_reward if self.expected_reward[key][0] < self.min_choose]
        if len(keys_of_less_selected)!=0:
            action = int(np.random.choice(keys_of_less_selected, 1))
            return action
        else:
            max_val = max(self.expected_reward[x][2] for x in self.expected_reward)
            keys_of_max_expected = [key for key in self.expected_reward if self.expected_reward[key][2] == max_val]
            action = int(np.random.choice(keys_of_max_expected))
            return action    
        
    def act(self):
        if np.random.choice([1, 0], p=[self.epsilon, 1-self.epsilon]):
            action = int(np.random.choice(range(action_space_size)))
        else:
            action = self.act_greedy()
        return action
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    def __init__(self, action_space_size, min_choose=1, initial_play_count=0, expected_reward=None):
        
            
    def act(self):
        keys_of_less_selected = [key for key in self.expected_reward if self.expected_reward[key][0] < self.min_choose]
        if len(keys_of_less_selected)!=0:
            action = int(np.random.choice(keys_of_less_selected, 1))
            return action
        else:
            max_val = max(self.expected_reward[x][2] for x in self.expected_reward)
            keys_of_max_expected = [key for key in self.expected_reward if self.expected_reward[key][2] == max_val]
            action = int(np.random.choice(keys_of_max_expected))
            return action
        
    def update(self, action, reward):
        self.play_count += 1
        self.expected_reward[action][0] += 1
        self.expected_reward[action][1] += reward
        self.expected_reward[action][2] = self.expected_reward[action][1] / self.expected_reward[action][0]

    def load_record(self, initial_play_count, expected_reward):
        self.play_count = initial_play_count
        if expected_reward is None or expected_reward=={}:
            self.expected_reward = {}
            for i in range(action_space_size):
                self.expected_reward[i] = [0, 0., 0.]
        else:
            self.expected_reward = expected_reward
        
    def save_record(self):
        return self.play_count, self.expected_reward

    def reset_record(self):
        self.play_count = 0
        self.expected_reward = {}
        for i in range(action_space_size):
            self.expected_reward[i] = [0, 0., 0.]
    
    def sum_reward(self):
        sum_reward = 0
        for value in self.expected_reward.values():
            sum_reward += value[1]
        return sum_reward
    
    def debug_sum(self):
        for action in self.expected_reward:
            print("self.expected_reward[{0}]".format(action), self.expected_reward[action])
        print("sum_reward", self.sum_reward())
        #print("sum_reward_fix", self.sum_reward_fix())
    
    def action_count(self):
        action_count = []
        for value in self.expected_reward.values():
            action_count.append(value[0])
        return action_count

### Q学習 : 方策オフ型TD制御
SARSAでは次状態$s'$を観測した後に$Q$から導かれる方策により行動$a'$を選択したが、  
Q学習では$\max_{a'}Q(s',a')$を与える$a'$を用いる。  

Algorithm
>$Q(s,a)$を任意に初期化
>各エピソードに対して繰り返し：  
>　　$s$を初期化    
>　　エピソードの各ステップに対して繰り返し：  
>　　　　$Q$から導かれる方策（εグリーディ方策など）を用いて、$s$で取る行動$a$を選択する  
>　　　　行動$a$を取り、報酬$r$と次状態$s'$を観測する  
>　　　　$Q(s,a) \leftarrow Q(s,a) + \alpha[r+\gamma \max_{a'} Q(s',a')-Q(s,a)]$  
>　　　　$s \leftarrow s';$  
>　　$s$が終端状態ならば繰り返しを終了 